In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\dy\modle\autogluon_type_class_DY_20240726"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

文件夹已存在


In [3]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\dy\train_data\train_20240726.csv")

In [4]:
dataset.columns

Index(['OBJECTID', 'DLMC', '母质', 'TL', 'YL', 'TS', 'TZ', 'XMin', 'YMin',
       'XMax',
       ...
       'DZ_MAX', 'DZ_RANGE', 'DZ_MEAN', 'DZ_STD', 'DZ_SUM', 'DZ_VARIETY',
       'DZ_MAJORITY', 'DZ_MINORITY', 'DZ_MEDIAN', 'DZ_PCT90'],
      dtype='object', length=433)

In [5]:
dataset.rename(columns={'母质': 'MZ'}, inplace=True)

In [6]:
dataset["TZ_label"] = dataset.TZ.astype("category").cat.codes

In [7]:
dataset['DLMC'] = dataset['DLMC'].astype('category')
dataset['MZ'] = dataset['MZ'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [8]:
result = dataset.groupby('TZ_label', observed=True)["TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层壤质中性紫色土', 1: '中层壤质酸性紫色土', 2: '中层壤质黄色石灰土', 3: '中层泥砂质黄壤', 4: '中层泥质黄壤', 5: '中层灰泥质黄壤', 6: '中层灰泥质黄色石灰土', 7: '中层砂泥质黄壤', 8: '中层砂泥黄壤', 9: '中层砾壤质黑色石灰土', 10: '中层硅质酸性粗骨土', 11: '中层红泥质黄壤', 12: '中层黏质黄色石灰土', 13: '厚层泥质黄壤', 14: '厚层灰泥质黄壤', 15: '厚层砂泥质黄壤', 16: '厚层硅质酸性粗骨土', 17: '厚层红泥质黄壤', 18: '厚层黏质黄色石灰土', 19: '浅石灰泥田', 20: '渗潮泥田', 21: '渗石灰泥田', 22: '潮泥田', 23: '石灰泥田', 24: '紫泥田', 25: '腐中层壤质中性紫色土', 26: '腐中层壤质酸性紫色土', 27: '腐中层壤质黄色石灰土', 28: '腐中层泥砂质黄壤', 29: '腐中层泥质黄壤', 30: '腐中层灰泥质黄壤', 31: '腐中层砂泥质黄壤', 32: '腐中层砂质山地灌丛草甸土', 33: '腐中层砾壤质黑色石灰土', 34: '腐中层硅质酸性粗骨土', 35: '腐中层硅质黄壤', 36: '腐中层红泥质黄壤', 37: '腐中层黏质黄色石灰土', 38: '腐厚层泥质黄壤', 39: '腐厚层灰泥质黄壤', 40: '腐厚层砂泥质黄壤', 41: '腐厚层硅质酸性粗骨土', 42: '腐厚层硅质黄壤', 43: '腐厚层红泥质黄壤', 44: '腐厚层黏质黄色石灰土', 45: '腐薄层壤质酸性紫色土', 46: '腐薄层壤质黄色石灰土', 47: '腐薄层壤质黑色石灰土', 48: '腐薄层泥砂质黄壤', 49: '腐薄层泥质黄壤', 50: '腐薄层灰泥质黄壤', 51: '腐薄层砂泥质黄壤', 52: '腐薄层砂质山地灌丛草甸土', 53: '腐薄层砾泥质黄壤性土', 54: '腐薄层硅质酸性粗骨土', 55: '腐薄层硅质黄壤', 56: '腐薄层硅质黄壤性土', 57: '腐薄层红泥质黄壤', 58: '腐薄层黏质黄色石灰土', 59: '薄层壤质酸性紫色土', 60: '薄层壤质黑色石灰土', 61: '薄层泥质黄壤', 62: '薄层灰泥质黄

In [10]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict_20240726.json', 'w') as f:
    json.dump(result, f)

In [9]:
# 查看地类类别
dl_df = pd.DataFrame(dataset['DLMC'].value_counts())

In [10]:
dl_df

,count
DLMC,
乔木林地,10523
水田,3563
灌木林地,2115
旱地,1419
其他草地,663
茶园,172
其他园地,78
其他林地,78
果园,59


In [11]:
# 查看母质类别
mz_df = pd.DataFrame(dataset['MZ'].value_counts())

In [12]:
mz_df

,count
MZ,
砂岩,5666
泥(页)岩,5512
碳酸岩,4922
第四系红粘土,1213
砂页岩,438
紫红色砂页岩,409
砾岩,276
河流冲积物,167
第四纪冰川冲积物,133


### 砂岩土种

In [21]:
# 筛选砂岩水稻土数据
sy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '砂岩')]
pd.unique(sy_sdt_data['TZ'])

array(['潮泥田', '黄青砂泥田', '黄渗白粉泥田', '黄白粉泥田', '黄浅白粉泥田', '黄鳝泥田', '黄青白粉泥田',
       '黄潮泥田'], dtype=object)

In [22]:
# 筛砂岩选非自然土数据
sy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '砂岩')]
pd.unique(sy_fzrt_data['TZ']),pd.unique(sy_fzrt_data['TS'])

(array(['薄层硅质黄壤', '厚层硅质酸性粗骨土'], dtype=object),
 array(['硅质黄壤', '硅质酸性粗骨土'], dtype=object))

In [23]:
# 筛选自然土数据
sy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '砂岩')]
pd.unique(sy_zrt_data['TZ']),pd.unique(sy_zrt_data['TS'])

(array(['腐厚层硅质黄壤', '腐薄层硅质黄壤', '腐中层砂质山地灌丛草甸土', '腐薄层硅质黄壤性土', '腐薄层砂质山地灌丛草甸土',
        '腐中层硅质黄壤', '薄层硅质黄壤'], dtype=object),
 array(['硅质黄壤', '砂质山地灌丛草甸土', '硅质黄壤性土'], dtype=object))

### 碳酸岩土种

In [24]:
# 筛选碳酸岩水稻土数据
tsy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '碳酸岩')]
pd.unique(tsy_sdt_data['TZ']),pd.unique(tsy_sdt_data['TS']),

(array(['石灰泥田', '青石灰泥田', '潮泥田', '渗潮泥田', '黄渗灰泥田', '浅石灰泥田', '黄浅灰泥田', '黄黄灰泥田',
        '渗石灰泥田'], dtype=object),
 array(['灰泥田', '青灰泥田', '潮泥田', '渗潮泥田', '渗灰泥田', '浅灰泥田', '黄灰泥田'], dtype=object))

In [17]:
# 筛选碳酸岩选非自然土数据
tsy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ( dataset['TZ'].str.contains('灰')) & (dataset['MZ'] != '泥(页)岩') &(dataset['MZ'] != '第四系红粘土')]
pd.unique(tsy_fzrt_data['TZ']),pd.unique(tsy_fzrt_data['TS']),

(array(['薄层砾壤质黑色石灰土', '中层灰泥质黄色石灰土', '中层灰泥质黄壤', '薄层灰泥质黄色石灰土', '中层砾壤质黑色石灰土',
        '中层壤质黄色石灰土', '薄层灰泥质黄壤', '厚层灰泥质黄壤', '薄层壤质黑色石灰土', '中层黏质黄色石灰土'],
       dtype=object),
 array(['壤质黑色石灰土', '灰泥质黄色石灰土', '灰泥质黄壤', '壤质黄色石灰土', '黏质黄色石灰土'], dtype=object))

In [18]:
# 筛选碳酸岩自然土数据
tsy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & ( dataset['TZ'].str.contains('灰')) & (dataset['MZ'] != '泥(页)岩') &(dataset['MZ'] != '第四系红粘土')]
pd.unique(tsy_zrt_data['TZ']),pd.unique(tsy_zrt_data['TS']),

(array(['腐薄层壤质黄色石灰土', '腐中层壤质黄色石灰土', '腐中层灰泥质黄壤', '腐薄层灰泥质黄壤', '腐薄层壤质黑色石灰土',
        '腐厚层灰泥质黄壤', '腐中层砾壤质黑色石灰土'], dtype=object),
 array(['壤质黄色石灰土', '灰泥质黄壤', '壤质黑色石灰土'], dtype=object))

### 第四系红粘土

In [19]:
# 筛选第四系红粘土水稻土数据
hnt_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '第四系红粘土')]
pd.unique(hnt_sdt_data['TZ']),pd.unique(hnt_sdt_data['TS']),

(array(['黄红泥田', '黄渗红泥田', '黄浅红泥田', '潮泥田', '黄青红泥田'], dtype=object),
 array(['红泥田', '渗红泥田', '浅红泥田', '潮泥田', '青红泥田'], dtype=object))

In [20]:
# 筛选第四系红粘土非自然土数据
hnt_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '第四系红粘土')]
pd.unique(hnt_fzrt_data['TZ']),pd.unique(hnt_fzrt_data['TS']),

(array(['薄层黏质黄色石灰土', '中层黏质黄色石灰土', '中层红泥质黄壤', '厚层红泥质黄壤', '厚层黏质黄色石灰土',
        '薄层红泥质黄壤'], dtype=object),
 array(['黏质黄色石灰土', '红泥质黄壤'], dtype=object))

In [25]:
# 筛选第四系红粘土自然土数据
hnt_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '第四系红粘土')]
pd.unique(hnt_zrt_data['TZ']),pd.unique(hnt_zrt_data['TS']),

(array(['腐厚层红泥质黄壤', '腐中层黏质黄色石灰土', '腐厚层黏质黄色石灰土', '腐中层红泥质黄壤', '腐薄层黏质黄色石灰土',
        '腐薄层红泥质黄壤'], dtype=object),
 array(['红泥质黄壤', '黏质黄色石灰土'], dtype=object))

### 泥(页)岩

In [26]:
# 筛选泥(页)岩水稻土数据
nyy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '泥(页)岩')]
pd.unique(nyy_sdt_data['TZ']),pd.unique(nyy_sdt_data['TS']),

(array(['潮泥田', '黄浅鳝泥田', '黄鳝泥田', '黄渗鳝泥田'], dtype=object),
 array(['潮泥田', '浅鳝泥田', '鳝泥田', '渗鳝泥田'], dtype=object))

In [27]:
# 筛选泥(页)岩非自然土数据
nyy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '泥(页)岩')]
pd.unique(nyy_fzrt_data['TZ']),pd.unique(nyy_fzrt_data['TS']),

(array(['薄层泥质黄壤', '厚层泥质黄壤', '中层泥质黄壤', '薄层黏质黄色石灰土'], dtype=object),
 array(['泥质黄壤', '黏质黄色石灰土'], dtype=object))

In [29]:
# 筛选泥(页)岩自然土数据
nyy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '泥(页)岩')]
pd.unique(nyy_zrt_data['TZ']),pd.unique(nyy_zrt_data['TS']),

(array(['腐薄层泥质黄壤', '腐厚层泥质黄壤', '腐薄层黏质黄色石灰土', '腐中层泥质黄壤', '腐中层黏质黄色石灰土',
        '腐厚层黏质黄色石灰土', '腐薄层砾泥质黄壤性土'], dtype=object),
 array(['泥质黄壤', '黏质黄色石灰土'], dtype=object))

### 紫红色砂页岩	

In [30]:
# 筛选紫红色砂页岩水稻土数据
zhsyy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & ((dataset['MZ'] == '紫红色砂页岩') | (dataset['MZ'] == '河流冲积物'))]
pd.unique(zhsyy_sdt_data['TZ']),pd.unique(zhsyy_sdt_data['TS']),

(array(['潮泥田', '渗潮泥田', '青潮泥田', '紫泥田'], dtype=object),
 array(['潮泥田', '渗潮泥田', '青潮泥田', '紫泥田'], dtype=object))

In [31]:
# 筛选紫红色砂页岩非自然土数据
zhsyy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '紫红色砂页岩')]
pd.unique(zhsyy_fzrt_data['TZ']),pd.unique(zhsyy_fzrt_data['TS']),

(array(['中层壤质酸性紫色土', '薄层壤质酸性紫色土', '中层壤质中性紫色土'], dtype=object),
 array(['壤质酸性紫色土', '壤质中性紫色土'], dtype=object))

In [32]:
# 筛选紫红色砂页岩自然土数据
zhsyy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '紫红色砂页岩')]
pd.unique(zhsyy_zrt_data['TZ']),pd.unique(zhsyy_zrt_data['TS']),

(array(['腐薄层壤质酸性紫色土', '腐中层壤质酸性紫色土', '腐中层壤质中性紫色土'], dtype=object),
 array(['壤质酸性紫色土', '壤质中性紫色土'], dtype=object))

### 河流冲积物

In [34]:
# 筛选河流冲积物水稻土数据
hlcjw_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) & dataset['TZ'].str.contains('潮泥田')]
pd.unique(hlcjw_sdt_data['TZ']),pd.unique(hlcjw_sdt_data['TS']),

(array(['潮泥田', '渗潮泥田', '青潮泥田', '黄潮泥田'], dtype=object),
 array(['潮泥田', '渗潮泥田', '青潮泥田', '黄潮泥田'], dtype=object))

### 砾岩

In [35]:
# 筛选砾岩水稻土数据
ly_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & ((dataset['MZ'] == '砾岩') | (dataset['MZ'] == '砂岩'))]
pd.unique(ly_sdt_data['TZ']),pd.unique(ly_sdt_data['TS']),

(array(['黄浅白粉泥田', '黄白粉泥田', '潮泥田', '黄青砂泥田', '黄渗白粉泥田', '黄鳝泥田', '黄青白粉泥田',
        '黄潮泥田'], dtype=object),
 array(['浅白粉泥田', '白粉泥田', '潮泥田', '青砂泥田', '渗白粉泥田', '鳝泥田', '青白粉泥田', '黄潮泥田'],
       dtype=object))

In [36]:
# 筛选砾岩非自然土数据
ly_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ((dataset['MZ'] == '砾岩') | (dataset['MZ'] == '砂岩'))]
pd.unique(ly_fzrt_data['TZ']),pd.unique(ly_fzrt_data['TS']),

(array(['厚层硅质酸性粗骨土', '中层硅质酸性粗骨土', '薄层硅质黄壤'], dtype=object),
 array(['硅质酸性粗骨土', '硅质黄壤'], dtype=object))

In [37]:
# 筛选砾岩非自然土数据
ly_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地']) 
                       & ((dataset['MZ'] == '砾岩') | (dataset['MZ'] == '砂岩'))]
pd.unique(ly_zrt_data['TZ']),pd.unique(ly_zrt_data['TS']),

(array(['腐厚层硅质酸性粗骨土', '腐厚层硅质黄壤', '腐中层硅质酸性粗骨土', '腐薄层硅质酸性粗骨土', '腐薄层硅质黄壤',
        '腐中层砂质山地灌丛草甸土', '腐薄层硅质黄壤性土', '腐薄层砂质山地灌丛草甸土', '腐中层硅质黄壤', '薄层硅质黄壤'],
       dtype=object),
 array(['硅质酸性粗骨土', '硅质黄壤', '砂质山地灌丛草甸土', '硅质黄壤性土'], dtype=object))

### 第四纪冰川冲积物

In [38]:
# 筛选第四纪冰川冲积物水稻土数据
bccjw_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                     ]
pd.unique(bccjw_sdt_data['TZ']),pd.unique(bccjw_sdt_data['TS']),

(array(['黄红泥田', '黄渗红泥田', '黄浅红泥田', '潮泥田', '黄青红泥田', '渗潮泥田', '青潮泥田', '黄浅白粉泥田',
        '黄白粉泥田', '黄浅鳝泥田', '黄鳝泥田', '黄渗鳝泥田', '黄青砂泥田', '黄渗白粉泥田', '黄青白粉泥田',
        '黄潮泥田', '黄砂泥田', '重漂砂泥田', '石灰泥田', '青石灰泥田', '黄渗灰泥田', '浅石灰泥田',
        '黄浅灰泥田', '黄黄灰泥田', '渗石灰泥田', '紫泥田'], dtype=object),
 array(['红泥田', '渗红泥田', '浅红泥田', '潮泥田', '青红泥田', '渗潮泥田', '青潮泥田', '浅白粉泥田',
        '白粉泥田', '浅鳝泥田', '鳝泥田', '渗鳝泥田', '青砂泥田', '渗白粉泥田', '青白粉泥田', '黄潮泥田',
        '砂泥田', '漂砂泥田', '灰泥田', '青灰泥田', '渗灰泥田', '浅灰泥田', '黄灰泥田', '紫泥田'],
       dtype=object))

In [62]:
# 筛选第四纪冰川冲积物非自然土数据
bccjw_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & ((dataset['MZ'] == '第四纪冰川冲积物') | (dataset['MZ'] == '砂岩'))]
pd.unique(bccjw_fzrt_data['TZ']),pd.unique(bccjw_fzrt_data['TS']),

(array(['中层泥砂质黄壤', '薄层硅质黄壤', '厚层硅质酸性粗骨土'], dtype=object),
 array(['泥砂质黄壤', '硅质黄壤', '硅质酸性粗骨土'], dtype=object))

In [41]:
# 筛选第四纪冰川冲积物自然土数据
bccjw_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '第四纪冰川冲积物')]
pd.unique(bccjw_zrt_data['TZ']),pd.unique(bccjw_zrt_data['TS']),

(array(['腐中层泥砂质黄壤', '腐薄层泥砂质黄壤'], dtype=object), array(['泥砂质黄壤'], dtype=object))

### 砂页岩

In [42]:
# 筛选砂页岩水稻土数据
syy_sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                      & (dataset['MZ'] == '砂页岩')]
pd.unique(syy_sdt_data['TZ']),pd.unique(syy_sdt_data['TS']),

(array(['黄砂泥田', '重漂砂泥田'], dtype=object), array(['砂泥田', '漂砂泥田'], dtype=object))

In [43]:
# 筛选砂页岩非自然土数据
syy_fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂']) 
                       & (dataset['MZ'] == '砂页岩')]
pd.unique(syy_fzrt_data['TZ']),pd.unique(syy_fzrt_data['TS']),

(array(['中层砂泥质黄壤', '薄层砂泥质黄壤', '中层砂泥黄壤', '薄层砂泥黄壤', '厚层砂泥质黄壤'], dtype=object),
 array(['砂泥质黄壤', '砂泥黄壤'], dtype=object))

In [44]:
# 筛选砂页岩物自然土数据
syy_zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])
                   & (dataset['MZ'] == '砂页岩')]
pd.unique(syy_zrt_data['TZ']),pd.unique(syy_zrt_data['TS']),

(array(['腐中层砂泥质黄壤', '腐薄层砂泥质黄壤', '腐厚层砂泥质黄壤'], dtype=object),
 array(['砂泥质黄壤'], dtype=object))

In [45]:
for i in dataset.columns:
    print(i)

OBJECTID
DLMC
MZ
TL
YL
TS
TZ
XMin
YMin
XMax
YMax
Centroid_X
Centroid_Y
DEM_COUNT
DEM_AREA
DEM_MIN
DEM_MAX
DEM_RANGE
DEM_MEAN
DEM_STD
DEM_SUM
DEM_MEDIAN
DEM_PCT90
AnalyticalHillshading_COUNT
AnalyticalHillshading_AREA
AnalyticalHillshading_MIN
AnalyticalHillshading_MAX
AnalyticalHillshading_RANGE
AnalyticalHillshading_MEAN
AnalyticalHillshading_STD
AnalyticalHillshading_SUM
AnalyticalHillshading_MEDIAN
AnalyticalHillshading_PCT90
Aspect_COUNT
Aspect_AREA
Aspect_MIN
Aspect_MAX
Aspect_RANGE
Aspect_MEAN
Aspect_STD
Aspect_SUM
Aspect_MEDIAN
Aspect_PCT90
ChannelNetworkBaseLevel_COUNT
ChannelNetworkBaseLevel_AREA
ChannelNetworkBaseLevel_MIN
ChannelNetworkBaseLevel_MAX
ChannelNetworkBaseLevel_RANGE
ChannelNetworkBaseLevel_MEAN
ChannelNetworkBaseLevel_STD
ChannelNetworkBaseLevel_SUM
ChannelNetworkBaseLevel_MEDIAN
ChannelNetworkBaseLevel_PCT90
ChannelNetworkDistance_COUNT
ChannelNetworkDistance_AREA
ChannelNetworkDistance_MIN
ChannelNetworkDistance_MAX
ChannelNetworkDistance_RANGE
ChannelNetworkD

In [46]:
# 定义要检查的关键字列表
keywords = ['DEM','ndvi', 'PCA_0', 'Slope','Aspect', 'MRVBF','TopographicWetnessIndex', 'Mean',  'PH','PRE','SRA','TMP','VAP','WIND','PlanCurvature']

# 定义包含字符串的列表
strings = list(dataset.columns)

# 使用列表推导式和any函数检查并添加到新列表
filtered_strings = [s for s in strings if any(keyword in s for keyword in keywords)]
print(len(filtered_strings))
print(filtered_strings)


160
['DEM_COUNT', 'DEM_AREA', 'DEM_MIN', 'DEM_MAX', 'DEM_RANGE', 'DEM_MEAN', 'DEM_STD', 'DEM_SUM', 'DEM_MEDIAN', 'DEM_PCT90', 'Aspect_COUNT', 'Aspect_AREA', 'Aspect_MIN', 'Aspect_MAX', 'Aspect_RANGE', 'Aspect_MEAN', 'Aspect_STD', 'Aspect_SUM', 'Aspect_MEDIAN', 'Aspect_PCT90', 'MRVBF_COUNT', 'MRVBF_AREA', 'MRVBF_MIN', 'MRVBF_MAX', 'MRVBF_RANGE', 'MRVBF_MEAN', 'MRVBF_STD', 'MRVBF_SUM', 'MRVBF_MEDIAN', 'MRVBF_PCT90', 'PlanCurvature_COUNT', 'PlanCurvature_AREA', 'PlanCurvature_MIN', 'PlanCurvature_MAX', 'PlanCurvature_RANGE', 'PlanCurvature_MEAN', 'PlanCurvature_STD', 'PlanCurvature_SUM', 'PlanCurvature_MEDIAN', 'PlanCurvature_PCT90', 'RelativeSlopePosition_COUNT', 'RelativeSlopePosition_AREA', 'RelativeSlopePosition_MIN', 'RelativeSlopePosition_MAX', 'RelativeSlopePosition_RANGE', 'RelativeSlopePosition_MEAN', 'RelativeSlopePosition_STD', 'RelativeSlopePosition_SUM', 'RelativeSlopePosition_MEDIAN', 'RelativeSlopePosition_PCT90', 'Slope_COUNT', 'Slope_AREA', 'Slope_MIN', 'Slope_MAX', 'Slop

In [57]:

sdt_features= ['DLMC','MZ','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_STD',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
fzrt_features = ['DLMC','MZ','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_STD',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
zrt_features = ['DLMC','MZ','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_STD',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
target = "TZ_label"

In [47]:
terms = [
    "Centroid_X",
    "Centroid_Y",
    "DEM_RANGE",
    "DEM_MEAN",
    "DEM_STD",
    "AnalyticalHillshading_MEAN",
    "Aspect_MEAN",
    "ChannelNetworkBaseLevel_MEAN",
    "ChannelNetworkDistance_MEAN",
    "ConvergenceIndex_MEAN",
    "LSFactor_MEAN",
    "MRRTF_MEAN",
    "MRVBF_MEAN",
    "PlanCurvature_MEAN",
    "ProfileCurvature_MEAN",
    "RelativeSlopePosition_MEAN",
    "Slope_MEAN",
    "TopographicWetnessIndex_MEAN",
    "TotalCatchmentArea_MEAN",
    "ValleyDepth_MEAN",
    "NIGHT2022_MEAN",
    "ETP2022_mean_MEAN",
    "TMP2022_mean_MEAN",
    "PRE2022_mean_MEAN",
    "PRE2022_3_MEAN",
    "PRE2022_11_MEAN",
    "ETP2022_3_MEAN",
    "ETP2022_11_MEAN",
    "TMP2022_3_MEAN",
    "TMP2022_11_MEAN",
    "evi_MEAN",
    "lswi_MEAN",
    "mndwi_MEAN",
    "ndmi_MEAN",
    "ndvi_MEAN",
    "ndwi_MEAN",
    "PCA_0_MEAN",
    "PCA_1_MEAN",
    "savi_MEAN",
    "vari_MEAN",
    "DL_MAJORITY",
    "SlopeClass_MAJORITY",
    "DZ_MAJORITY"
]


In [48]:

sdt_features= ['DLMC','MZ']+ terms
fzrt_features = ['DLMC','MZ']+ terms
zrt_features = ['DLMC','MZ']+ terms
target = "TZ_label"

### 砂岩数据集

In [49]:
# 获取sdt数据集
sy_sdt_data = sy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
sy_fzrt_data = sy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
sy_zrt_data = sy_zrt_data[zrt_features+[target]]

### 碳酸岩数据集

In [50]:
# 获取sdt数据集
tsy_sdt_data = tsy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
tsy_fzrt_data = tsy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
tsy_zrt_data = tsy_zrt_data[zrt_features+[target]]

### 第四系红粘土数据集

In [51]:
# 获取sdt数据集
hnt_sdt_data = hnt_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
hnt_fzrt_data = hnt_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
hnt_zrt_data = hnt_zrt_data[zrt_features+[target]]

### 泥(页)岩数据集

In [52]:
# 获取sdt数据集
nyy_sdt_data = nyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
nyy_fzrt_data = nyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
nyy_zrt_data = nyy_zrt_data[zrt_features+[target]]

### 紫红色砂页岩数据集

In [53]:
# 获取sdt数据集
zhsyy_sdt_data = zhsyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
zhsyy_fzrt_data = zhsyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
zhsyy_zrt_data = zhsyy_zrt_data[zrt_features+[target]]

### 河流冲积物数据集

In [54]:
# 获取sdt数据集
hlcjw_sdt_data = hlcjw_sdt_data[sdt_features+[target]]

### 砾岩数据集

In [55]:
# 获取sdt数据集
ly_sdt_data = ly_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
ly_fzrt_data = ly_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
ly_zrt_data = ly_zrt_data[zrt_features+[target]]

### 第四纪冰川冲积物数据集

In [63]:
# 获取sdt数据集
bccjw_sdt_data = bccjw_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
bccjw_fzrt_data = bccjw_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
bccjw_zrt_data = bccjw_zrt_data[zrt_features+[target]]

### 砂页岩数据集

In [57]:
# 获取sdt数据集
syy_sdt_data = syy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
syy_fzrt_data = syy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
syy_zrt_data = syy_zrt_data[zrt_features+[target]]

In [58]:
# 查看各个数据集的大小
print('砂岩',sy_sdt_data.shape,sy_fzrt_data.shape,sy_zrt_data.shape)
print('碳酸岩',tsy_sdt_data.shape,tsy_fzrt_data.shape,tsy_zrt_data.shape)
print('第四系红粘土',hnt_sdt_data.shape,hnt_fzrt_data.shape,hnt_zrt_data.shape)
print('泥页岩',nyy_sdt_data.shape,nyy_fzrt_data.shape,nyy_zrt_data.shape)
print('紫红色砂页岩',zhsyy_sdt_data.shape,zhsyy_fzrt_data.shape,zhsyy_zrt_data.shape)
print('河流冲积物',hlcjw_sdt_data.shape)
print('砾岩',ly_sdt_data.shape,ly_fzrt_data.shape,ly_zrt_data.shape)
print('第四纪冰川冲积物',bccjw_sdt_data.shape,bccjw_fzrt_data.shape,bccjw_zrt_data.shape)
print('砂页岩',syy_sdt_data.shape,syy_fzrt_data.shape,syy_zrt_data.shape)


砂岩 (2494, 46) (68, 46) (3104, 46)
碳酸岩 (310, 46) (763, 46) (3849, 46)
第四系红粘土 (379, 46) (205, 46) (629, 46)
泥页岩 (100, 46) (160, 46) (5252, 46)
紫红色砂页岩 (195, 46) (146, 46) (235, 46)
河流冲积物 (268, 46)
砾岩 (2553, 46) (112, 46) (3277, 46)
第四纪冰川冲积物 (3563, 46) (32, 46) (101, 46)
砂页岩 (26, 46) (332, 46) (80, 46)


In [59]:
# 确定标签
label = "TZ_label"
eval_metric = 'f1_weighted'
problem_type = 'multiclass'
# 初始化模型和超参数
hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


### 模型训练

In [60]:
def train_model(train_data,label,eval_metric,hyperparameters,problem_type,model_master,model_branch,model_root):
    # 训练model
    temp_master_path =os.path.join(model_root,model_master)
    temp_branch_path = os.path.join(temp_master_path,model_branch)
    # 检查路径是否存在，否则便创建
    if not os.path.exists(temp_branch_path):
        os.makedirs(temp_branch_path)
    else:
        print("文件夹已存在")
    # 执行训练
    train_predictor = TabularPredictor(label=label,path=temp_branch_path,eval_metric=eval_metric,problem_type=problem_type).fit(train_data,hyperparameters=hyperparameters)
    train_predictor.fit_summary()
    return train_predictor.model_best


### 模型训练

In [ ]:
# 砂岩
sy_sdt_predictor = train_model(sy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='sdt',model_root=model_path) 
sy_fzrt_predictor = train_model(sy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='fzrt',model_root=model_path) 
sy_zrt_predictor = train_model(sy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='zrt',model_root=model_path) 
# 碳酸岩
tsy_sdt_predictor = train_model(tsy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='sdt',model_root=model_path) 
tsy_fzrt_predictor = train_model(tsy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='fzrt',model_root=model_path) 
tsy_zrt_predictor = train_model(tsy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='zrt',model_root=model_path) 
# 第四系红粘土
hnt_sdt_predictor = train_model(hnt_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='sdt',model_root=model_path) 
hnt_fzrt_predictor = train_model(hnt_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='fzrt',model_root=model_path) 
hnt_zrt_predictor = train_model(hnt_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='zrt',model_root=model_path) 
# 泥页岩
nyy_sdt_predictor = train_model(nyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='sdt',model_root=model_path) 
nyy_fzrt_predictor = train_model(nyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='fzrt',model_root=model_path) 
nyy_zrt_predictor = train_model(nyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='zrt',model_root=model_path) 
# 紫红色砂页岩
zhsyy_sdt_predictor = train_model(zhsyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='sdt',model_root=model_path) 
zhsyy_fzrt_predictor = train_model(zhsyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='fzrt',model_root=model_path) 
zhsyy_zrt_predictor = train_model(zhsyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='zrt',model_root=model_path) 
# 河流冲积物
hlcjw_sdt_predictor = train_model(hlcjw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='sdt',model_root=model_path)
# 砾岩
ly_sdt_predictor = train_model(ly_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='ly',model_branch='sdt',model_root=model_path) 
ly_fzrt_predictor = train_model(ly_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='ly',model_branch='fzrt',model_root=model_path) 
ly_zrt_predictor = train_model(ly_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='ly',model_branch='zrt',model_root=model_path) 
# 第四纪冰川冲积物
bccjw_sdt_predictor = train_model(bccjw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='bccjw',model_branch='sdt',model_root=model_path) 
bccjw_fzrt_predictor = train_model(bccjw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='bccjw',model_branch='fzrt',model_root=model_path) 
bccjw_zrt_predictor = train_model(bccjw_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='bccjw',model_branch='zrt',model_root=model_path) 
# 砂页岩
syy_sdt_predictor = train_model(syy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='sdt',model_root=model_path) 
syy_fzrt_predictor = train_model(syy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='fzrt',model_root=model_path) 
syy_zrt_predictor = train_model(syy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='zrt',model_root=model_path) 